In [1]:
import requests
import json
from pprint import pprint
from fake_useragent import UserAgent
import  pandas as pd

ua = UserAgent().chrome

# Request

In [2]:
data = [[524], 270881,1]
r = requests.post('https://www.codingame.com/services/Puzzle/findProgressByIds', data = str(data))
r.text

'[]'

In [3]:
full_data = [list(range(1, 524)), 270881,1]
r = requests.post('https://www.codingame.com/services/Puzzle/findProgressByIds',  data = str(full_data))

In [4]:
pd.DataFrame(json.loads(r.text)).to_csv("results_raw.csv", index=False)

# Load & Cleanup

In [5]:
import pandas as pd
import json
import ast

def compute_total(x):
    json_data = ast.literal_eval(x)
    return sum(json_data["feedbacks"])

def compute_average(x):
    json_data = ast.literal_eval(x)
    total = 0
    for pts, vote in zip([1, 2, 3, 4, 5], json_data["feedbacks"]):
        total += pts * vote
    return total / sum(json_data["feedbacks"])   

In [62]:
df = pd.read_csv("results_raw.csv", encoding = "ISO-8859-1")

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 40 columns):
achievementCount        400 non-null int64
attemptCount            400 non-null int64
chatRoom                61 non-null object
codinGamer              240 non-null object
communityCreation       400 non-null bool
contributor             313 non-null object
coverBinaryId           122 non-null float64
creationTime            400 non-null int64
description             68 non-null object
detailsPageUrl          400 non-null object
doneAchievementCount    400 non-null int64
feedback                400 non-null object
forumLink               167 non-null object
globalTotal             43 non-null float64
id                      400 non-null int64
lastActivity            233 non-null float64
league                  11 non-null object
level                   400 non-null object
logoBinaryId            87 non-null float64
mode                    11 non-null object
openChallenge           

In [8]:
# df.testSessionHandle

In [9]:
df['creationTime'] = pd.to_datetime(df['creationTime']*1000000)
df['lastActivity'] = pd.to_datetime(df['lastActivity']*1000000)
       
df["feedbackAvg"] = df["feedback"].apply(compute_average)
df["feedbackCount"] = df["feedback"].apply(compute_total)

df.drop(["chatRoom", "codinGamer", "contributor", 
         "coverBinaryId", "forumLink", "league", 
         "logoBinaryId", "openChallenge", "optimCriteria", 
         "optimCriteriaId", "optimPoints", "prettyId", 
         "previewBinaryId", "puzzleLeaderboardId", "rank", 
         "replayIds", "score", "thumbnailBinaryId",
         "topics", "feedback"], axis=1, inplace=True)  # "testSessionHandle"

df = df.set_index("id")
df = df.sort_index()

In [10]:
df.to_csv("results.csv")

# Analysis

In [49]:
import pandas as pd

In [50]:
df = pd.read_csv("results.csv", encoding = "ISO-8859-1", index_col=0)

In [51]:
df["base"] = "https://www.codingame.com"
df["url"] = df["base"].str.cat(df["detailsPageUrl"])

In [4]:
df.head()

,achievementCount,attemptCount,communityCreation,creationTime,description,detailsPageUrl,doneAchievementCount,globalTotal,lastActivity,level,...,position,solvedCount,testSessionHandle,title,total,type,validatorScore,xpPoints,feedbackAvg,feedbackCount
id,,,,,,,,,,,,,,,,,,,,,
1,3,55181,False,2014-07-16 13:25:18.000000000,Le virus de la résistance se propage à travers...,/training/medium/skynet-revolution-episode-1,3,NaN,2019-07-26 15:34:22.688000000,medium,...,NaN,31377,37819577b3990cd63492da5e8688b35cfa3b7f5,Skynet Revolution - Episode 1,NaN,SOLO,100,150.0,4.732390,3038
4,2,370635,False,2014-07-23 08:13:21.000000000,"Le marteau de Thor, <i>Mjöllnir</i> a perdu to...",/training/easy/power-of-thor-episode-1,2,NaN,2018-08-03 15:02:46.048999936,easy,...,NaN,233421,37542667842fae110bd7bc244b4eebc7927c25c,Power of Thor - Episode 1,NaN,SOLO,100,50.0,4.437905,17739
5,2,190857,False,2014-07-23 08:14:24.000000000,Il fait froid ! Saurez-vous trouver la tempéra...,/training/easy/temperatures,2,NaN,2017-07-15 08:56:55.223000064,easy,...,NaN,136543,3760408937c17934059ee020ae0f137f92a4ba9,Températures,NaN,CODE,100,50.0,4.235134,12882
6,2,109833,False,2014-07-23 08:16:54.000000000,Vous-vous sentez un âme d'artiste ? Mais vous ...,/training/easy/ascii-art,2,NaN,2017-09-17 09:41:33.336999936,easy,...,NaN,73647,37612433492b02b8dfb1d5ce584f10b1aa174dd,ASCII Art,NaN,CODE,100,50.0,4.433223,7278
7,2,86561,False,2014-07-23 08:19:26.000000000,La binaire c'est bien ! Mais le unaire c'est m...,/training/easy/chuck-norris,2,NaN,2018-08-13 15:54:40.916999936,easy,...,NaN,63004,3761504deb84534c75a88ff4325125ce6109a7f,Chuck Norris,NaN,CODE,100,50.0,4.261606,5428


In [6]:
# df[df.achievementCount > df.doneAchievementCount]

In [23]:
# df[df.total<df.globalTotal]

In [39]:
df["percentDone"] = df.solvedCount / df.attemptCount

In [40]:
sub = df[df.communityCreation]
sub = sub[sub.validatorScore < 100]
sub.sort_values(['percentDone'], ascending=[0])

,achievementCount,attemptCount,communityCreation,creationTime,description,detailsPageUrl,doneAchievementCount,globalTotal,lastActivity,level,...,position,solvedCount,title,total,type,validatorScore,xpPoints,feedbackAvg,feedbackCount,percentDone
id,,,,,,,,,,,,,,,,,,,,,
448,0,120,True,2018-12-17 01:04:01.182000000,NaN,/training/medium/ip-mask-calculating,0,NaN,NaN,medium,...,NaN,111,IP mask calculating,NaN,CODE,0,50.0,4.000000,30,0.925000
519,0,24,True,2019-07-13 16:24:56.337000000,NaN,/training/medium/a-childs-play,0,NaN,NaN,medium,...,NaN,22,A child's play,NaN,CODE,0,50.0,4.111111,9,0.916667
417,0,70,True,2018-08-24 12:38:04.794000000,NaN,/training/hard/dumbbells-solver,0,NaN,NaN,hard,...,NaN,64,Dumbbells solver,NaN,CODE,0,50.0,4.500000,10,0.914286
401,0,161,True,2018-06-26 16:59:59.992000000,NaN,/training/medium/trits-balanced-ternary-computing,0,NaN,NaN,medium,...,NaN,145,Trits (Balanced Ternary Computing),NaN,CODE,0,50.0,4.424242,33,0.900621
421,0,158,True,2018-09-17 17:44:00.363000000,NaN,/training/medium/kaprekars-routine,0,NaN,NaN,medium,...,NaN,139,Kaprekar's routine,NaN,CODE,0,50.0,4.324324,37,0.879747
173,0,492,True,2016-07-13 12:38:10.257000000,NaN,/training/medium/queneau-numbers,0,NaN,NaN,medium,...,NaN,430,Queneau Numbers,NaN,CODE,0,50.0,4.137931,58,0.873984
456,0,85,True,2019-01-18 07:25:21.949000000,NaN,/training/medium/green-valleys,0,NaN,NaN,medium,...,NaN,74,Green Valleys,NaN,CODE,0,50.0,4.318182,22,0.870588
394,0,166,True,2018-06-17 20:57:24.862000000,NaN,/training/medium/simplified-monopoly-turns-pre...,0,NaN,NaN,medium,...,NaN,143,Simplified Monopoly? Turns Prediction,NaN,CODE,0,50.0,4.406250,32,0.861446
400,0,191,True,2018-06-26 11:26:41.371000000,NaN,/training/medium/bulgarian-solitaire,0,NaN,NaN,medium,...,NaN,164,Bulgarian solitaire,NaN,CODE,0,50.0,4.321429,28,0.858639


# Write README.md

In [52]:
from tabulate import tabulate

In [54]:
df["url"] = df["url"].apply(lambda x:"[Link]({})".format(x))

In [59]:
def to_cat(x):
    pts = {
        "medium" : 2,
        "hard" : 3,
        "easy" : 1,
        "expert" : 4,
        "multi" : 7,
        "optim" : 6,
        "codegolf" : 5,
        "tutorial" : 0
    }
    return pts[x]

df["diff"] = df["level"].apply(to_cat)

In [56]:
def to_emojis(x):
    if x==100:
        return ":heavy_check_mark:"  
    elif x>0: 
        return ":white_check_mark:" 
    else: 
        return ""

df["Done"] = df["validatorScore"].apply(to_emojis)

In [65]:
df = df.sort_values(["diff", "validatorScore"], ascending=[True, False])

In [68]:
sub = df[["title", "level", "Done", "url"]]
sub = sub.set_index("title")
txt = tabulate(sub, tablefmt="pipe", headers="keys")

In [70]:
with open("README.md", "w") as f:
    f.write(txt)

# Download solutions

In [12]:
import json
import requests
from pprint import pprint

In [13]:
cookies = {"remcg" : "9N7aF+pmVPnTDnw1Lu8C4yVd3uJMkR8pcMchRiBrIqbfrGx8oD3Te1k1wW1CDm5RcrPl5JFNeFXrFEzUcBRDMPKEgw0kvyTmR4RbIyjtsLV65L7qj+E5Vry5PH2ge5s6X2qnOcIgJ9tOaSyXZsEtJxm7AeTpyq+1IufHeQffK9NvMXM2vz8/H66o5ggjv6XZWzxNlvQVlUIqO48H7sA5IzwKNL6+dBVleErp56aVMop8cWyjG+rBr24Fj4IvBh/fDnSZ7Tmt+EUO67arJHhcgd0gK/pU5La04jjKRTd6/Wbrw8LI488L6/sf8byNp8JE93XZzLNBOopFlzzrV5SzSfIMj1wZ1+Lzt+wICx5yBzaAf4RjI3jCNETrrMrZfbRI5fMsuPfNhNcmcIcs8N3FvduftGUPwLHhgcJKg/GLRRBNi4+2KMFW/jWyMKVajj5zKi+QTZTF4f8x8eNGvgEGhz5/avvObZJ51N1EMozsODifJddf5Wp2m5lR4Xyhr2D4+NI+oK4QqK2eiYyEqFbRFioeNGYUokG+OV4b0OqynVuGRs5cNkJlbp2TtbPM+Y4Y/0mKQ46TLrXCyPr/UOZbyg==",
           "driftt_aid" : "b93bbc9b-60b8-476b-b81e-f262a19ca711", 
           "G_ENABLED_IDPS" : "google", 
           "rememberMe":"2708813e52ef597fc6a811d3c8f2e5b34de9b5",
           "cgSession":"c002a793-9a20-44c7-9a68-6a10d602e435",
           "fbm_165331130336219":"base_domain=.www.codingame.com",
           "AWSALB":"6Htea83xg8vVLtJYCbLnrUAD7dEx4rJ7SK5IDjHqUl61cbEnxjj376B0HlxZuRvXNIMz7JNTNw4sWzZkIN+iMCMtkJio933EkIRHneNq4KUNMyw3e/BB7uVUnSzeq6CxpIPu7cIRg1HFeXKRmyRM6zHYcNbA+xpddxR61d5Wapx6Bx+9qlIJYuFkJXRHDg==",
           "amplitude_id_ecc0b9f76710f07825ed40270d4623e5codingame.com":"eyJkZXZpY2VJZCI6IjMyNzYzNWFiLWEyYTgtNDg0MC04ZDJjLTI5ZDRjMTdhM2M2YVIiLCJ1c2VySWQiOiIyNzA4ODEiLCJvcHRPdXQiOmZhbHNlLCJzZXNzaW9uSWQiOjE1NjQyMDk2OTY3MTQsImxhc3RFdmVudFRpbWUiOjE1NjQyMDk3MTc1ODQsImV2ZW50SWQiOjI1MjcsImlkZW50aWZ5SWQiOjMzNTksInNlcXVlbmNlTnVtYmVyIjo1ODg2fQ=="}

In [38]:
# data = ["20730215e9a4091099765b3747c71c253e7a89c3"]
# data = [df.loc[5, "testSessionHandle"]]
# r = requests.post('https://www.codingame.com/services/TestSession/startTestSession', cookies=cookies, data = str(data))
# r.text
# js = json.loads(r.text)
# pprint(js)

In [41]:
# code = js["currentQuestion"]["answer"]["code"]
# language = js["currentQuestion"]["answer"]["programmingLanguageId"]

In [22]:
# data = ["20730215e9a4091099765b3747c71c253e7a89c3", "Python3"]
# data = [df.loc[5, "testSessionHandle"], "Python3"]
# r = requests.post('https://www.codingame.com/services/TestSession/getPreviousCodeByLanguageId', cookies=cookies, data = str(data))
# r.text
# js = json.loads(r.text)
# pprint(js)

In [15]:
def extract_code(testSessionHandle, language="Python3"):
    data = [testSessionHandle, language]
    r = requests.post('https://www.codingame.com/services/TestSession/getPreviousCodeByLanguageId', cookies=cookies, data = str(data))
    code = json.loads(r.text)
    return code

In [24]:
import os
for row in df.iterrows():
    testSessionHandle = row[1]["testSessionHandle"]
    validatorScore = row[1]["validatorScore"]
    level = row[1]["level"]
    root, folder, title = row[1]["detailsPageUrl"][1:].split("/")
    title = title.replace(":", "")
    if validatorScore == 0:
        continue
    extension = {
        "Python3" : "py",
        "Javascript" : "js",
    }
    language = "Python3"
    filename = "{}/{}/{}.{}".format(root, folder, title, extension[language])
    if os.path.isfile(filename):
        continue
    if not os.path.exists("{}/{}".format(root, folder)):
        os.makedirs("{}/{}".format(root, folder))
    print("Extracting {} : {}".format(testSessionHandle, filename))
    code = extract_code(testSessionHandle, language=language)
    if code is None:
        continue
    with open(filename, "w") as f : 
        f.write(code)

Extracting 44767464569782efab981a76a28c38a35e74d6b : training/medium/ascii-art--glass-stacking.py
